# Sampling (RA, dec)

TDAstro provides multiple mechanisms for sampling (RA, dec). In this notebook we discuss several of the approaches and their relative tradeoffs.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from tdastro.math_nodes.ra_dec_sampler import (
    OpSimRADECSampler,
    OpSimUniformRADECSampler,
    UniformRADEC,
)
from tdastro.opsim.opsim import OpSim

## Uniform Sampling

The simplest sampling approach is to uniformly sample (RA, dec) from the unit sphere. The `UniformRADEC` node does exactly this.

In [ ]:
uniform_sampler = UniformRADEC(node_label="uniform")
(ra, dec) = uniform_sampler.generate(num_samples=500)

plt.scatter(ra, dec, s=1)
plt.show()

However this approach has limited use when simulating a specific survey. Depending on the survey's coverage, a significant number of (RA, dec) points may fall outside the viewing area.

## Sampling from a Survey

We can sample (RA, dec) coordinates from a survey (an `OpSim` object) in two ways. First we could sample a pointing from the survey and then a point from that field of view. Second, we could sample uniformly from the region coverage by the survey.

We consider each of these approaches below.

### Sampling Pointings

Sampling pointings from the survey provides a visit-weighted sampling of positions covered by the survey. For concreteness let's start with a survey that visits two fields: one centered at (45.0, -15.0) and the other at (315.0, 15.0). The first field is visited once and the second field is visited four times on four consecutive nights.

In [ ]:
values = {
    "observationStartMJD": np.array([0.0, 1.0, 2.0, 3.0, 4.0]),
    "fieldRA": np.array([45.0, 315.0, 315.0, 315.0, 315.0]),
    "fieldDec": np.array([-15.0, 15.0, 15.0, 15.0, 15.0]),
    "zp_nJy": np.ones(5),
}
opsim = OpSim(values, radius=30.0)

We can sample from these pointings using the `OpSimRADECSampler` node.

In [ ]:
pointing_sampler = OpSimRADECSampler(opsim, radius=30.0, node_label="opsim")
(ra, dec, time) = pointing_sampler.generate(num_samples=100)

plt.scatter(ra, dec, s=1)
plt.show()

As we can see, the field centered in the Northern hemisphere is sampled significantly more than the one centered in the Southern hemisphere.

### Sampling Coverage

If we instead would like to sample uniformly from the area covered by the survey, we have two options. The `OpSimUniformRADECSampler` uses rejection sampling to generate positions. This approach can be slow for surveys with small coverage. However, since it requires no pre-computation, it is a viable option for generating a small number of samples from a survey with high coverage.

In [ ]:
coverage_sampler1 = OpSimUniformRADECSampler(opsim, radius=30.0, node_label="coverage1")
(ra, dec) = coverage_sampler1.generate(num_samples=100)

plt.scatter(ra, dec, s=1)
plt.show()

## Linking (RA, dec) to Sources

To be useful, the (RA, dec) locations that we generate must be linked into our source objects. To support this all the generators above produce a pair of named outputs "ra" and "dec". This means we can use TDAstro's reference functionality to set the source's position based on the samples.

In [ ]:
from tdastro.sources.basic_sources import StaticSource

source = StaticSource(
    ra=uniform_sampler.ra,
    dec=uniform_sampler.dec,
    brightness=100.0,
    node_label="source",
)
state = source.sample_parameters(num_samples=10)
print(state)